## Bisection Method

For a cheat sheet on Jupyter notebook's markdown language, see https://www.ibm.com/support/knowledgecenter/SSHGWL_1.2.3/analyze-data/markd-jupyter.html

Finding the midpoint of two numbers (the arithmetic average) seems like a simple task, but it's not. The simple-minded formula $\frac{a+b}{2}$ can needlessly overflow, and then often suggested alternative (for example, _Numerical Analysis_  by Richard L. Burden  and  J. Douglas Faires)  $a + \frac{b-a}{2}$ has other problems, including needless overflow when $a$ and $b$ have opposite signs.

We can avoid the needless overflow of $\frac{a+b}{2}$ by using $\frac{a}{2}+ \frac{b}{2}$, but this formula can underflow to zero, giving a midpoint that is not in between $a$ and $b$.

For more details than you will want to know, see https://hal.archives-ouvertes.fr/file/index/docid/576641/filename/computing-midpoint.pdf


For rational inputs, we'll use rational arithmetic to find the midpoint; for all other numeric types,we'll use the simple minded $\frac{a+b}{2}$. Julia's multiple function dispatch makes it possible to have multiple definitions of a function with distinct input types.



In [ ]:
function midpoint(a::Rational,b::Rational)
    (a+b)//2
end   

In [ ]:
function midpoint(a::Number,b::Number)
    (a+b)/2
end  

In the bisection method, traditionally you see a test $fa \times fm > 0$ (see, for example, see \https://mmas.github.io/bisection-method-julia}. But  $fa \times fm$ could _needlessly_ overflow. 

Instead we'll test using a function same\_sign\_p (the final p in the function name is for "predicate,'' meaning a function that returns either true or false). This method won't needlessly overflow plus it uses fewer resources. 

To show that it uses fewer resources, we need run the tests more than once.  This is due to the fact that Julia uses a just in time compiler. The first time the test gets run, it gets compiled.

In [ ]:
function same_sign_p(x,y) 
    (x < 0 && y <0) || (x > 0 && y >0)
end

In [ ]:
x, y = rand(), -rand();

In [ ]:
@time x*y > 0

In [ ]:
@time x*y > 0

In [ ]:
@time same_sign_p(y,x)

In [ ]:
@time same_sign_p(x,y)

For our bisection method code, we need to test if two numbers are approximately equal. We'll use the Julia function isapprox to to this. The user documentation is

In [ ]:
? isapprox

The times for both methods are about the same, but the function same\_sign\_p allocates less memory than does using $x y > 0$.  We should protect against endless (or nearly endless loops)--we'll do this with a limit on the number of bisections.

Here is our code for the bisection method. The method stops when the zero has been located to be in an interval $[a,b]$ with 
$$
  |a - b|  \leq  \mathrm{tol}  \times (1 + \max(|a|, |b|).
$$
 The Julia function isapprox does this test.
 
Maybe this code should special case fm == 0. Another day. We'll default the tolerance to the magic number 64 times the binary64 machine epsilon; and default the maximum number of iterations to the magic number 1000. 

In [ ]:
function bisection(f::Function,a::Number,b::Number, tol = 64*eps(Float64), maxiter::Integer = 1000)
    fa = f(a)
    fb = f(b)
    # Error when f has the same sign at endpoionts a & b.
    !same_sign_p(fa,fb) || error("Bisection: Function doesn't have opposite signs at given endpoints.")
               
    while !isapprox(a,b,atol = tol,rtol = tol) && maxiter > 0
        maxiter -= 1
        m = midpoint(a,b)
        fm = f(m)
        if same_sign_p(fa, fm) 
            (a,fa) = (m, fm) # the zero of f is between m & b
        else
            (b,fb) = (m, fm) # the zero of f is between a & m
        end       
    end    
    # error when too many iterations; otherwise, return the midpoint
    if maxiter <= 0
        error("Bisection: Exceeded maximum number of iterations.")
    else     
       midpoint(a,b)
    end
end;

In [ ]:
bisection(x -> x^2 - 2,0, 2)


Fun: we can use BigInt rational numbers too!

In [ ]:
bisection(x -> x^2 - 2,BigInt(0)//1, BigInt(2)//1, 1//BigInt(2)^107)

Our method doesn't assume an ordering of the inputs:

In [ ]:
bisection(x -> x^2 - 2,2,0)

In [ ]:
bisection(x -> x^2 - 2,-2,0)

Let's test the mechanism for detecting an error

In [ ]:
bisection(x -> x^2 - 2, 5,107)

And test the mechanism for limiting the number of bisections

In [ ]:
bisection(x -> x^2 - 2, 0.0, 1.0e308)

In [ ]:
using Gadfly

For a given number $y$, let's solve $x \exp(x) = y$. We start with a graphical analysis. The minimum of 
$x \mapsto x \exp(x)$ is $-1/e$. For $y \in [-1/e,0) $, the equation $x \exp(x) = y$ has two real solutions; and for $ y \in [0,\infty)$, there is only one real solution.

In [ ]:
plot(x -> x*exp(x),-5,0.5)

The minimum of this function is $-1/e $ (and this happens when the input is -1).

Specifically, look at a graph of $x  \mapsto  x \exp(x)  + 1/10$.  One zero is between -4 and -3, and the other is between -1 and 0. Let's use the bisection method to find both real solutions.

In [ ]:
plot(x -> x*exp(x)  + 0.1,-5,0.5)

In [ ]:
bisection(x -> x*exp(x) + 0.1,-0.5,1)

In [ ]:
bisection(x -> x*exp(x) + 0.1,-4.0,-3.0)

Actually, this function has a name--it is the _Lambert W function_. The domain  of principal Lambert W function is 
$[-1/e, \infty) $. A bisection method for evaluating the Lambert W function is

In [ ]:
function lambert_w(x::Float64)
    if x < -1/MathConstants.e
        throw(DomainError(x))
    elseif x < 0
         bisection(q -> q*exp(q)-x,-1.0, 0.0)
    elseif x == 0
        zero(x)
    else
      bisection(q -> q*exp(q)-x,0,x)
    end
end

In [ ]:
plot(lambert_w, -0.3,5)

The Lambert W function is useful in applied mathematics. This function is useful for finding the average blood sugar level given the percent of hemoglobin molecules that have formed chemical bounds with glucose (the HbA1C in medical speak), for example. And for lots of others uses too.

Don't trust everything (anything?) you find on the Internet. The first result for a search on "julia bisection method"  leads to https://mmas.github.io/bisection-method-julia. Here is that code

In [ ]:
function bisection_rubbish(f::Function, a::Number, b::Number;
                   tol::AbstractFloat=1e-5, maxiter::Integer=100)
    fa = f(a)
    fa*f(b) <= 0 || error("No real root in [a,b]")
    i = 0
    local c
    while b-a > tol
      
        i += 1
        i != maxiter || error("Max iteration exceeded")
        c = (a+b)/2
        fc = f(c)
        if fc == 0
            break
        elseif fa*fc > 0
            a = c  # Root is in the right half of [a,b].
            fa = fc
        else
            b = c  # Root is in the left half of [a,b].
        end
    end
    return c
end

This code assumes that $b > a$, but it doesn't check for that condition. If that condition isn't satisfied, it needlessly loops. Actually, it's worse than that. Outside the ``while'' loop, the variable "c" doesn't have a value. The programmer attempted to fix this using a "local" declaration. Maybe this once worked in Julia, but not now:

In [ ]:
bisection_rubbish(x -> x^2 - 2, 2,0)

In [ ]:
bisection_rubbish(x -> -10*(x - 1.0e158) + 1.0e158, 1.0e158, 2.0e158)

In [ ]:
bisection(x -> -10*(x - 1.0e158) + 1.0e158, 1.0e158, 2.0e158)